In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import matplotlib.pyplot as plt
%matplotlib inline
from multiprocess import Pool
import numpy as np

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *
from data_manager import *

from registration_utilities import *

Setting environment for Precision WorkStation


No vtk


In [2]:
atlas_spec = dict(name='atlasV6',
                   vol_type='score'    ,               
                    resolution='10.0um'
                   )

In [3]:
atlas_structures_wrt_canonicalAtlasSpace_atlasResol = \
DataManager.load_original_volume_all_known_structures_v3(atlas_spec, in_bbox_wrt='canonicalAtlasSpace',
                                                        out_bbox_wrt='canonicalAtlasSpace')

Prior structure/index map not found. Generating a new one.


In [4]:
atlas_structure_centroids_wrt_canonicalAtlasSpace_atlasResol = get_structure_centroids(vol_origin_dict=atlas_structures_wrt_canonicalAtlasSpace_atlasResol)

In [5]:
atlas_anchor1_wrt_canonicalAtlasSpace_atlasResol = \
np.r_[atlas_structure_centroids_wrt_canonicalAtlasSpace_atlasResol['12N'][:2], 0]
atlas_anchor2_wrt_canonicalAtlasSpace_atlasResol = \
np.r_[atlas_structure_centroids_wrt_canonicalAtlasSpace_atlasResol['3N_L'][:2], 0]

In [6]:
stack = 'CHATM2'

In [7]:
if stack == 'CHATM2':
    subject_anchor1_wrt_wholebrainWithMargin_atlasResol = np.array([565,246,179]) * 20. / 10. # 12N
    subject_anchor2_wrt_wholebrainWithMargin_atlasResol = np.array([388,183,179]) * 20. / 10. # 3N_L
elif stack == 'CHATM3':
    subject_anchor1_wrt_wholebrainWithMargin_atlasResol = np.array([740,258,230]) * 20. / 10.
    subject_anchor2_wrt_wholebrainWithMargin_atlasResol = np.array([550,195,230]) * 20. / 10.

In [8]:
def align_anchors(pm1, pm2, pf1, pf2):
    """
    Align the vector (pm1, pm2) and vector (pf1, pf2), and align the two middle points. 
    """
    
    t = ((pf1 + pf2) / 2. - (pm1 + pm2) / 2.)

    subject_d = pf1 - pf2
    atlas_d = pm1 - pm2

    subject_d_n = subject_d / np.linalg.norm(subject_d)
    atlas_d_n = atlas_d / np.linalg.norm(atlas_d)

    R = R_align_two_vectors(atlas_d_n, subject_d_n)

    T = np.zeros((3,4))
    T[:3, :3] = R
    T[:3, 3] = t
    
    return T

In [9]:
intensity_volume_spec = dict(name=stack, resolution='10.0um', prep_id='wholebrainWithMargin', vol_type='intensity')
_, wholebrainWithMargin_origin_wrt_wholebrain_dataResol = DataManager.load_original_volume_v2(intensity_volume_spec, return_origin_instead_of_bbox=True)

In [10]:
subject_anchor1_wrt_wholebrain_atlasResol = subject_anchor1_wrt_wholebrainWithMargin_atlasResol + \
wholebrainWithMargin_origin_wrt_wholebrain_dataResol # only valid because dataResol is same as atlasResol

subject_anchor2_wrt_wholebrain_atlasResol = subject_anchor2_wrt_wholebrainWithMargin_atlasResol + \
wholebrainWithMargin_origin_wrt_wholebrain_dataResol # only valid because dataResol is same as atlasResol

In [11]:
T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol = align_anchors(atlas_anchor1_wrt_canonicalAtlasSpace_atlasResol, atlas_anchor2_wrt_canonicalAtlasSpace_atlasResol,
             subject_anchor1_wrt_wholebrain_atlasResol, subject_anchor2_wrt_wholebrain_atlasResol)

In [12]:
print T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol

[[ 9.90767489e-01  1.35582009e-01  0.00000000e+00  9.22098379e+02]
 [-1.35582009e-01  9.90767489e-01  0.00000000e+00  4.50968802e+02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  3.58000000e+02]]


In [ ]:
bp.pack_ndarray_file(T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol, 
                     '/home/yuncong/' + stack + '_T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol.bp')